In [1]:
!git clone https://github.com/ultralytics/JSON2YOLO.git

Cloning into 'JSON2YOLO'...
remote: Enumerating objects: 394, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 394 (delta 226), reused 195 (delta 183), pack-reused 136
Receiving objects: 100% (394/394), 111.11 KiB | 8.55 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [2]:
import sys
sys.path.append('/kaggle/working/JSON2YOLO')

In [3]:
from general_json2yolo import convert_coco_json

In [4]:
convert_coco_json('/kaggle/input/livecell', use_segments=True)

Annotations /kaggle/input/livecell/livecell_coco_test.json: 100%|██████████| 1564/1564 [00:31<00:00, 50.40it/s] 
Annotations /kaggle/input/livecell/livecell_coco_train.json: 100%|██████████| 3253/3253 [01:08<00:00, 47.25it/s] 
Annotations /kaggle/input/livecell/livecell_coco_val.json: 100%|██████████| 570/570 [00:11<00:00, 48.57it/s] 


In [5]:
import json
import shutil
import os


def copy_images_from_coco_json(coco_json_file, source_dir, target_dir):
    with open(coco_json_file, 'r') as f:
        data = json.load(f)

    if 'images' not in data:
        print("Помилка: JSON файл не містить ключ 'images'")
        return

    images = data['images']
    
    for image_info in images:
        image_path = os.path.join(source_dir, image_info['file_name'])

        if os.path.exists(image_path):
            image_name = os.path.basename(image_path)
            target_path = os.path.join(target_dir, image_name)

            os.makedirs(target_dir, exist_ok=True)
            shutil.copy(image_path, target_path)
        else:
            print(f"Попередження: Зображення {image_path} не знайдено!")

    print("Процес копіювання завершено.")

coco_json_file = '/kaggle/input/livecell/livecell_coco_test.json'
source_directory = '/kaggle/input/livecell/images/'
target_directory = '/kaggle/working/copied_images/'

copy_images_from_coco_json('/kaggle/input/livecell/livecell_coco_test.json',
                           '/kaggle/input/livecell/images/images/livecell_test_images', 
                           '/kaggle/working/new_dir/images/livecell_coco_test')

copy_images_from_coco_json('/kaggle/input/livecell/livecell_coco_train.json',
                           '/kaggle/input/livecell/images/images/livecell_train_val_images', 
                           '/kaggle/working/new_dir/images/livecell_coco_train')

copy_images_from_coco_json('/kaggle/input/livecell/livecell_coco_val.json',
                           '/kaggle/input/livecell/images/images/livecell_train_val_images', 
                           '/kaggle/working/new_dir/images/livecell_coco_val')


Процес копіювання завершено.
Процес копіювання завершено.
Процес копіювання завершено.


In [16]:
import yaml

names = ['cell']

d = {
        "path": '/kaggle/working/new_dir',
        "train": '/kaggle/working/new_dir/images/livecell_coco_train',
        "val": "/kaggle/working/new_dir/images/livecell_coco_val",
        "test": "/kaggle/working/new_dir/images/livecell_coco_test",
        "nc": len(names),
        "names": names,
}  # dictionary

with open('/kaggle/working/new_dir/dataset.yaml', "w") as f:
    yaml.dump(d, f, sort_keys=False)

In [17]:
import shutil
import os
import sys
from colorama import Fore

In [18]:
class SetupPipline:
    def __init__(self, display: bool = True):
        self.pycocotools = self.__pycocotools()
        self.ultralytics = self.__ultralytics()
        
    @staticmethod
    def __ultralytics() -> str:
        sys.path.append("/kaggle/input/hubmap-tools-ultralytics-and-pycocotools/ultralytics/ultralytics") 
        return "successfully"
        
    @staticmethod
    def __pycocotools() -> str:
        if not os.path.exists("/kaggle/working/packages"):
            shutil.copytree("/kaggle/input/hubmap-tools-ultralytics-and-pycocotools/pycocotools/pycocotools", "/kaggle/working/packages")
            os.chdir("/kaggle/working/packages/pycocotools-2.0.6/")
            os.system("python setup.py install")
            os.system("pip install . --no-index --find-links /kaggle/working/packages/")
            os.chdir("/kaggle/working")
            return "successfully"
    
    def display(self) -> None:
        print(Fore.GREEN+f"\nPycocotools was installed {self.pycocotools}")
        print(f"Ultralytics was installed {self.ultralytics}"+Fore.WHITE)

In [19]:
pipline = SetupPipline()

In [20]:
pipline.display()


Pycocotools was installed None
Ultralytics was installed successfully


In [21]:
from pycocotools import _mask as coco_mask 
from ultralytics import YOLO

In [22]:
def main():
    model = YOLO("yolov8x-seg.pt")
    model.train(
        project="live-cell",
        name="yolov8x-seg",

        deterministic=True,
        seed=43,

        data="/kaggle/working/new_dir/dataset.yaml", 
        save=True,
        save_period=5,
        pretrained=True,
        imgsz=512,

        epochs=20,
        batch=4,
        workers=8,
        val=True,
        device=0,

        lr0=0.018,
        patience=3,
        optimizer="SGD",
        momentum=0.947,
        weight_decay=0.0005,
        close_mosaic=3,
    )

In [ ]:
main()

New https://pypi.org/project/ultralytics/8.2.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.120 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=/kaggle/working/new_dir/dataset.yaml, epochs=20, patience=3, batch=4, imgsz=512, save=True, save_period=5, cache=False, device=0, workers=8, project=live-cell, name=yolov8x-seg, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=43, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=3, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnost

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
100%|██████████| 6.23M/6.23M [00:00<00:00, 107MB/s]
AMP: checks passed ✅
train: Scanning /kaggle/working/new_dir/labels/livecell_coco_train... 3188 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3188/3188 [00:32<00:00, 97.17it/s] 
train: WARNING ⚠️ /kaggle/working/new_dir/images/livecell_coco_train/BV2_Phase_D4_2_02d12h00m_4.tif: 2 duplicate labels removed
train: WARNING ⚠️ /kaggle/working/new_dir/images/livecell_coco_train/BV2_Phase_D4_2_02d16h00m_4.tif: 1 duplicate labels removed
train: New cache created: /kaggle/working/new_dir/labels/livecell_coco_train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely l